In [18]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import pandas as pd
import math

In [19]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):   # n_folds = 5 given
    #print('len of data', len(dataset))
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)    
    #print('foldsize',fold_size)
    for i in range(n_folds):
        #print('i', i)
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            #print('index',index)
            fold.append(dataset_copy.pop(index))
            #print('len of dataset_copy after popping', len(dataset_copy))
            #print('fold',fold)
        dataset_split.append(fold)
        #print('datasplit',dataset_split)
        #print('len of datasplit',len(dataset_split))
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [20]:
#Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    #print('**********in evaluate algo*************')
    folds = cross_validation_split(dataset, n_folds)   # taking 5 cross validatoin data of containing 40 rows each from dataset 
    #print('crossfolds',len(folds))
    scores = list()
    for fold in folds:
        #print('fold',len(fold))    # iterating through each fold in 5 folds
        train_set = list(folds)
        train_set.remove(fold)  # remove that fold from train_set 
        train_set = sum(train_set, [])  #  sum(iterable, start) , iterable can be list tupple or dictionaries ,
                                        #  but most importantly it should be numbers. adding empty list to train_set list
        test_set = list()
        #print("train", len(train_set))
        
        for row in fold:       # iterating each row in that fold
            #print('row',row)
            row_copy = list(row)
            test_set.append(row_copy)  # add the rows to test_set from that fold
            row_copy[-1] = None
        #print("test", len(test_set)) 
        tree, predicted = algorithm(train_set, test_set, *args)
        #print('tree', tree)
        #print('predict', predicted)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return tree, scores


In [50]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    #print("********* in test_split******")
    #print('value', value)
    for row in dataset:
        #print('row[index]', row[index])
        #print('len of row', len(row))
        if row[index] < value:
            left.append(row)
            #print('left', left)
        else:
            right.append(row)
            #print('right', right)
    #print('left', len(left))
    #print('right', len(right))
    return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    #print("**************in gini_index**************")
    #print("n_instance", n_instances)   #  give 164.0 sum of (120 + 44)
    # sum weighted Gini index for each group
    gini = 0.0
    #print('group',groups)
    for group in groups:
        size = float(len(group))
        #print('len of group', size)  # will give the size of left and right groups i.e. 120.0(left) &  44.0(right) 
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            #print('class value', class_val)
            p = [row[-1] for row in group].count(class_val) / size  # counting frequency of the class values in the group
            #print('p', [row[-1] for row in group].count(class_val))
            score += p * p
        # weight the group score by its relative size
        #print('score',score)
        gini += (1.0 - score) * (size / n_instances)
    #print('gini', gini)
    return gini

def cross_entropy(groups, classes):
    #print('********* cross *********')
    n_instances = float(sum([len(group) for group in groups]))
    #print('instances', n_instances)
    cross_entropy = 0.0
    for group in groups:
        size = float(len(group))
        #print('size', size)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            #print('p', p)
            if p == 0:
                score = 0
            else:
                score+= -p * math.log2(p)
            #print('score', score)
        cross_entropy += score * (size / n_instances)
        #print('score', score)
    return cross_entropy

def total_entropy(data_set):
    #print('********* total*********')
    size = len(dataset)
    class_value = list(set(row[-1] for row in dataset))
    #print('class', class_value)
    entropy = 0.0
    score = 0.0
    for class_val in class_value:
        p = [row[-1] for row in dataset].count(class_val) / size
        #print('p', p)
        score+= -p * math.log2(p) 
    return score    
    


# Select the best split point for a dataset
# def get_split(dataset, n_features):
#     #print("******** in get split **********")
#     class_values = list(set(row[-1] for row in dataset))  # set the class values for last column of the row = {0,1}
#     #print('calss values',class_values)
#     b_index, b_value, b_score, b_groups = 999, 999, 999, None
#     features = list()
#     while len(features) < n_features:    # until len of feature < 7
#         index = randrange(len(dataset[0])-1)  # generate the random index in range 60
#         #print('index', index)
#         #print('features', features)
#         if index not in features:
#             features.append(index)
#     for index in features:    # iterate the index numbers(48,52,...) in feature i.e. features [48, 52, 54, 46, 14, 34]
#         for row in dataset:
#             groups = test_split(index, row[index], dataset)  # will give left and right group of that index column in row
#             gini = gini_index(groups, class_values)  # will  give the gini index using left, right groups and class values
#             if gini < b_score:
#                 b_index, b_value, b_score, b_groups = index, row[index], gini, groups
#     return {'index':b_index, 'value':b_value, 'groups':b_groups}


# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 0.0, 0.0, 0.0, None
    data_entropy = total_entropy(dataset)
    #print('dataentropy', data_entropy)
    #print('********* ****************')
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            cross_entr = cross_entropy(groups, class_values)
            entropy = data_entropy - cross_entr
            #print('cross_entropy', entropy)
            gini = gini_index(groups, class_values)
            if entropy > b_score:
                b_index, b_value, b_score, b_groups = index, row[index], entropy, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

  
 
# Make a prediction with a decision tree
def predict(node, row):
    #print('node:-', node['index'])
    #print('node value:-', node['value'])
    if row[node['index']] < node['value']:  
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
 
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):  #ratio = 1.0 given
    sample = list()
    n_sample = round(len(dataset) * ratio)   # len of dataset = 208
    while len(sample) < n_sample:           #  len(sample) < 208 till than
        index = randrange(len(dataset))     # generate the random index value in range 0 - 208
        sample.append(dataset[index])       #  add the index row values to sample => subsample
    return sample
 
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)
 
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size)  # return root node 
        trees.append(tree)
    #print('tree', tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return trees, predictions



In [51]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [52]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    print(unique)
    for i, value in enumerate(unique):
        #print(value)
        #print(i)
        lookup[value] = i
        #print('for value', value,'-',lookup[value])
    for row in dataset:
        #print(row)
        row[column] = lookup[row[column]]
    return lookup

In [53]:
# split the alpha string to numeric data
def str_char_to_num(dataset,column):
    for row in dataset:
        #print('rowcolumn', type(row[column]))
        i = list((str(row[column]).split())[0])[0]
        row[column] = i
    '''#print('n',i) 
    if i == 'm':
        row[2] = 0
    else:
        row[2] = 1
    j = list((row[6].split())[0])[0]
    row[6] = j
    k = list((row[8].split())[0])[0]
    row[8] = k'''
    

In [54]:
filename = 'train.csv'
dataset = load_csv(filename)
df = pd.read_csv("train.csv",header=0)
cols = df.columns.tolist()
df = df[['PassengerId', 'Pclass','Sex', 'Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked', 'Survived']]
df.fillna(0,inplace=True)
print(type(df))
print(type(dataset))
data=df.values.tolist()
print(type(data))

<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'list'>


In [55]:
  
str_char_to_num(data, len(data[0])-2) 
str_char_to_num(data, len(data[0])-3) 
str_char_to_num(data, len(data[0])-5) 
str_char_to_num(data, len(data[0])-9)
str_column_to_int(data, len(data[0])-2) 
str_column_to_int(data, len(data[0])-3) 
str_column_to_int(data, len(data[0])-5) 
str_column_to_int(data, len(data[0])-9) 


{'Q', 'S', '0', 'C'}
{'B', 'D', 'E', 'G', 'A', '0', 'T', 'F', 'C'}
{'W', '8', '4', '5', '9', 'S', '7', 'A', '3', 'P', '6', 'L', 'F', '2', 'C', '1'}
{'f', 'm'}


{'f': 0, 'm': 1}

In [57]:
seed(2)
#evaluate algorithm
n_folds = 10
max_depth = 7
min_size = 5
sample_size = 1
n_features = int(sqrt(len(dataset[0])-1))
#print(n_features)
for n_trees in [10]:
    tree, scores = evaluate_algorithm(data, random_forest, n_folds, max_depth, min_size, sample_size, n_trees)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    #print('tree',tree)
    

Trees: 10
Scores: [82.02247191011236, 82.02247191011236, 86.51685393258427, 76.40449438202246, 80.89887640449437, 79.7752808988764, 84.26966292134831, 84.26966292134831, 77.52808988764045, 77.52808988764045]
Mean Accuracy: 81.124%


In [47]:
filename = 'test.csv'
testset = load_csv(filename)
df = pd.read_csv("test.csv",header=0)
cols = df.columns.tolist()
df = df[['PassengerId', 'Pclass', 'Sex', 'Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']]
df.fillna(0,inplace=True)
print(type(df))
print(type(testset))
test=df.values.tolist()
print(type(test))
for row in test:
    print('row', row)
    break

<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'list'>
row [892, 3, 'male', 34.5, 0, 0, '330911', 7.8292, 0, 'Q']


In [48]:
for row in test:
    print('row', row)
    break
    
str_char_to_num(test, len(test[0])-1) 
str_char_to_num(test, len(test[0])-2) 
str_char_to_num(test, len(test[0])-4) 
str_char_to_num(test, len(test[0])-8) 
str_column_to_int(test, len(test[0])-1) 
str_column_to_int(test, len(test[0])-2) 
str_column_to_int(test, len(test[0])-4) 
str_column_to_int(test, len(test[0])-8)

for row in test:
    print('row', row)
    
import csv
sub_data=[]
schema = ['PassengerId','Survived']
with open('submission_entropy.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow([g for g in schema]) 
    for row in test:
        print('row',row[0])
        test_predict = bagging_predict(tree, row)
        #print('predict', test_predict)
        sub_data = list([row[0], test_predict])
        print(sub_data)
        writer.writerow(sub_data)
f.close()
sub = pd.read_csv("submission.csv")
print(len(sub))        

row [892, 3, 'male', 34.5, 0, 0, '330911', 7.8292, 0, 'Q']
{'S', 'C', 'Q'}
{'B', 'D', 'E', 'A', 'G', '0', 'F', 'C'}
{'W', '4', '9', 'S', '7', 'A', '3', 'P', 'L', 'F', '6', 'C', '2', '1'}
{'f', 'm'}
row [892, 3, 1, 34.5, 0, 0, 6, 7.8292, 5, 2]
row [893, 3, 0, 47.0, 1, 0, 6, 7.0, 5, 0]
row [894, 2, 1, 62.0, 0, 0, 12, 9.6875, 5, 2]
row [895, 3, 1, 27.0, 0, 0, 6, 8.6625, 5, 0]
row [896, 3, 0, 22.0, 1, 1, 6, 12.2875, 5, 0]
row [897, 3, 1, 14.0, 0, 0, 4, 9.225, 5, 0]
row [898, 3, 0, 30.0, 0, 0, 6, 7.6292, 5, 2]
row [899, 2, 1, 26.0, 1, 1, 12, 29.0, 5, 0]
row [900, 3, 0, 18.0, 0, 0, 12, 7.2292, 5, 1]
row [901, 3, 1, 21.0, 2, 0, 5, 24.15, 5, 0]
row [902, 3, 1, 0.0, 0, 0, 6, 7.8958, 5, 0]
row [903, 1, 1, 46.0, 0, 0, 10, 26.0, 5, 0]
row [904, 1, 0, 23.0, 1, 0, 12, 82.2667, 0, 0]
row [905, 2, 1, 63.0, 1, 0, 12, 26.0, 5, 0]
row [906, 1, 0, 47.0, 1, 0, 0, 61.175, 2, 0]
row [907, 2, 0, 24.0, 1, 0, 3, 27.7208, 5, 1]
row [908, 2, 1, 35.0, 0, 0, 12, 12.35, 5, 2]
row [909, 3, 1, 21.0, 0, 0, 12, 7.225, 5

node:- 7
node value:- 25.4667
node:- 6
node value:- 8
node:- 0
node value:- 41
node:- 7
node value:- 9.8417
node:- 0
node value:- 730
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 3
node value:- 28.0
node:- 4
node value:- 3
node:- 7
node value:- 20.2125
node:- 7
node value:- 15.2458
node:- 0
node value:- 768
[898, 0]
row 899
node:- 2
node value:- 1
node:- 7
node value:- 26.0
node:- 6
node value:- 9
node:- 3
node value:- 51.0
node:- 8
node value:- 4
node:- 6
node value:- 13
node:- 0
node value:- 794
node:- 2
node value:- 1
node:- 7
node value:- 23.25
node:- 4
node value:- 3
node:- 3
node value:- 54.0
node:- 6
node value:- 8
node:- 0
node value:- 184
node:- 0
node value:- 218
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 6
node value:- 13
node:- 7
node value:- 24.15
node:- 0
node value:- 666
node:- 0
node value:- 666
node:- 2
node value:- 1
node:- 7
node value:- 26.0
node:- 0
node value:- 184
node:- 6
node value:- 9
node:- 4
node value:- 5
node:- 0
node value:- 742
node

node value:- 25.4667
node:- 0
node value:- 403
node:- 6
node value:- 8
node:- 0
node value:- 883
node:- 0
node value:- 883
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 23.45
node:- 6
node value:- 14
node:- 3
node value:- 17.0
node:- 0
node value:- 403
node:- 6
node value:- 5
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 0
node value:- 403
node:- 7
node value:- 21.075
node:- 6
node value:- 14
node:- 9
node value:- 1
node:- 0
node value:- 502
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 7
node value:- 7.75
node:- 0
node value:- 101
node:- 7
node value:- 15.5
node:- 3
node value:- 28.0
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 27.9
node:- 6
node value:- 8
node:- 4
node value:- 1
node:- 0
node value:- 41
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 6
node value:- 5
node:- 0
node value:- 148
node:- 0
node value:- 148
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node va

node:- 0
node value:- 742
node:- 0
node value:- 803
node:- 6
node value:- 15
node:- 2
node value:- 1
node:- 7
node value:- 15.2458
node:- 6
node value:- 9
node:- 3
node value:- 54.0
node:- 0
node value:- 573
node:- 8
node value:- 4
node:- 0
node value:- 699
node:- 2
node value:- 1
node:- 7
node value:- 26.2875
node:- 5
node value:- 1
node:- 6
node value:- 7
node:- 3
node value:- 32.0
node:- 3
node value:- 24.0
node:- 4
node value:- 2
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 6
node value:- 15
node:- 7
node value:- 21.0
node:- 7
node value:- 133.65
node:- 8
node value:- 7
node:- 0
node value:- 451
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 0
node value:- 166
node:- 7
node value:- 7.225
node:- 3
node value:- 0.67
node:- 0
node value:- 829
node:- 0
node value:- 838
[921, 0]
row 922
node:- 2
node value:- 1
node:- 7
node value:- 26.0
node:- 6
node value:- 9
node:- 3
node value:- 7.0
node:- 0
node value:- 103
node:- 0
node value:- 103
node:- 2
node value:- 1
node:- 7

node:- 5
node value:- 1
node:- 3
node value:- 11.0
node:- 9
node value:- 3
node:- 0
node value:- 60
node:- 0
node value:- 60
node:- 2
node value:- 1
node:- 6
node value:- 9
node:- 7
node value:- 7.75
node:- 7
node value:- 32.3208
node:- 8
node value:- 5
node:- 5
node value:- 2
node:- 7
node value:- 12.525
node:- 2
node value:- 1
node:- 7
node value:- 15.2458
node:- 6
node value:- 7
node:- 0
node value:- 221
node:- 7
node value:- 7.925
node:- 7
node value:- 8.3625
node:- 0
node value:- 243
node:- 2
node value:- 1
node:- 7
node value:- 26.2875
node:- 5
node value:- 1
node:- 3
node value:- 15.0
node:- 0
node value:- 623
node:- 0
node value:- 747
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 6
node value:- 15
node:- 7
node value:- 21.0
node:- 5
node value:- 1
node:- 3
node value:- 16.0
node:- 9
node value:- 3
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 0
node value:- 166
node:- 7
node value:- 7.225
node:- 3
node value:- 0.67
node:- 3
node value:- 4.0
node:- 3
node value

...

node:- 5
node value:- 1
node:- 0
node value:- 585
node:- 0
node value:- 745
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 0
node value:- 166
node:- 7
node value:- 7.225
node:- 3
node value:- 0.67
node:- 0
node value:- 829
node:- 0
node value:- 838
[1250, 0]
row 1251
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 0
node value:- 403
node:- 6
node value:- 8
node:- 0
node value:- 883
node:- 0
node value:- 883
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 23.45
node:- 6
node value:- 14
node:- 3
node value:- 17.0
node:- 0
node value:- 403
node:- 6
node value:- 5
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 0
node value:- 403
node:- 7
node value:- 21.075
node:- 6
node value:- 14
node:- 9
node value:- 1
node:- 0
node value:- 502
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 7
node value:- 7.75
node:- 0
node value:- 101
node:- 7
node value:- 15.5
node:- 0
node value:- 568
node

node value:- 9
node:- 7
node value:- 15.2458
node:- 0
node value:- 278
node:- 5
node value:- 1
node:- 3
node value:- 31.0
node:- 3
node value:- 36.0
node:- 2
node value:- 1
node:- 7
node value:- 15.2458
node:- 6
node value:- 7
node:- 3
node value:- 25.0
node:- 3
node value:- 34.5
node:- 7
node value:- 13.0
node:- 0
node value:- 669
node:- 2
node value:- 1
node:- 7
node value:- 26.2875
node:- 5
node value:- 1
node:- 6
node value:- 7
node:- 3
node value:- 32.0
node:- 0
node value:- 100
node:- 0
node value:- 318
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 0
node value:- 249
node:- 0
node value:- 746
node:- 0
node value:- 858
node:- 0
node value:- 868
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 0
node value:- 371
node:- 3
node value:- 58.0
node:- 6
node value:- 10
node:- 0
node value:- 749
node:- 5
node value:- 2
[1264, 0]
row 1265
node:- 2
node value:- 1
node:- 7
node value:- 26.0
node:- 5
node value:- 1
node:- 3
node value:- 27.0
node:- 3
node value:- 33.0
node:- 0


node:- 1
node value:- 3
node:- 7
node value:- 24.15
node:- 7
node value:- 15.2458
node:- 6
node value:- 8
node:- 0
node value:- 618
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 6
node value:- 8
node:- 0
node value:- 41
node:- 7
node value:- 9.8417
node:- 7
node value:- 16.1
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 3
node value:- 28.0
node:- 6
node value:- 8
node:- 0
node value:- 475
node:- 0
node value:- 618
node:- 0
node value:- 618
[1275, 0]
row 1276
node:- 2
node value:- 1
node:- 7
node value:- 26.0
node:- 5
node value:- 1
node:- 3
node value:- 27.0
node:- 0
node value:- 297
node:- 7
node value:- 7.775
node:- 0
node value:- 838
node:- 2
node value:- 1
node:- 7
node value:- 23.25
node:- 0
node value:- 806
node:- 0
node value:- 816
node:- 0
node value:- 816
node:- 2
node value:- 1
node:- 8
node value:- 5
node:- 6
node value:- 13
node:- 7
node value:- 24.15
node:- 0
node value:- 147
node:- 7
node value:- 20.525
node:- 6
node value:-

node:- 1
node value:- 3
node:- 7
node value:- 30.0
node:- 0
node value:- 582
node:- 0
node value:- 582
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 0
node value:- 319
node:- 3
node value:- 44.0
node:- 0
node value:- 330
node:- 0
node value:- 330
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 29.0
node:- 0
node value:- 538
node:- 0
node value:- 538
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 26.25
node:- 3
node value:- 4.0
node:- 0
node value:- 709
node:- 0
node value:- 709
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 26.25
node:- 0
node value:- 4
node:- 0
node value:- 4
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 3
node value:- 4.0
node:- 3
node value:- 38.0
node:- 0
node value:- 231
node:- 0
node value:- 231
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 8
node value:- 5
node:- 3
node value:- 25.0
node:- 3
node value:- 3.0
node:- 0
node value:- 317
node:- 0
node value:- 317
node:- 2


node:- 3
node value:- 28.0
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 7
node value:- 7.75
node:- 0
node value:- 157
node:- 0
node value:- 157
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 27.9
node:- 6
node value:- 8
node:- 4
node value:- 1
node:- 0
node value:- 236
node:- 7
node value:- 9.8417
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 6
node value:- 5
node:- 0
node value:- 397
node:- 0
node value:- 645
node:- 6
node value:- 8
node:- 0
node value:- 730
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 3
node value:- 18.0
node:- 0
node value:- 613
node:- 0
node value:- 778
node:- 0
node value:- 778
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 24.15
node:- 7
node value:- 15.2458
node:- 3
node value:- 28.0
node:- 7
node value:- 8.1375
node:- 7
node value:- 7.225
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 7
node value:- 25.4667
node:- 6
node